In [2]:
import numpy as np
import pandas as pd
import gc
import pickle

In [89]:
# DATASET A IMPORTAR
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
stores = pd.read_csv("stores.csv")
#sub = pd.read_csv("sample_submission.csv")   
transactions = pd.read_csv("transactions.csv").sort_values(["store_nbr", "date"])
oil = pd.read_csv("oil.csv")
holidays = pd.read_csv("holidays_events.csv")


# FECHAS Y HORAS
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)
holidays["date"] = pd.to_datetime(holidays.date)
oil["date"] = pd.to_datetime(oil.date)


# AJUSTE TIPOS DE DATOS OBJETO A NÚMERICO
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")


In [90]:
# Días festivos transferidos

tr1 = holidays[(holidays.type == "Holiday") & (holidays.transferred == True)].drop("transferred", axis = 1).reset_index(drop = True)
tr2 = holidays[(holidays.type == "Transfer")].drop("transferred", axis = 1).reset_index(drop = True)
tr = pd.concat([tr1,tr2], axis = 1)
tr = tr.iloc[:, [5,1,2,3,4]]

holidays = holidays[(holidays.transferred == False) & (holidays.type != "Transfer")].drop("transferred", axis = 1)
holidays = holidays.append(tr).reset_index(drop = True)

# Días festivos adicionales
holidays["description"] = holidays["description"].str.replace("-", "").str.replace("+", "").str.replace('\d+', '')
holidays["type"] = np.where(holidays["type"] == "Additional", "Holiday", holidays["type"])

# Días festivos "Puentes"
holidays["description"] = holidays["description"].str.replace("Puente ", "")
holidays["type"] = np.where(holidays["type"] == "Bridge", "Holiday", holidays["type"])

 
# Días festivos trabajados, esto para saldar el puente
work_day = holidays[holidays.type == "Work Day"]  
holidays = holidays[holidays.type != "Work Day"]

# Segmentación de datos

# Eventos Nacionales
events = holidays[holidays.type == "Event"].drop(["type", "locale", "locale_name"], axis = 1).rename({"description":"events"}, axis = 1)

holidays = holidays[holidays.type != "Event"].drop("type", axis = 1)
regional = holidays[holidays.locale == "Regional"].rename({"locale_name":"state", "description":"holiday_regional"}, axis = 1).drop("locale", axis = 1).drop_duplicates()
national = holidays[holidays.locale == "National"].rename({"description":"holiday_national"}, axis = 1).drop(["locale", "locale_name"], axis = 1).drop_duplicates()
local = holidays[holidays.locale == "Local"].rename({"description":"holiday_local", "locale_name":"city"}, axis = 1).drop("locale", axis = 1).drop_duplicates()



d = pd.merge(train.append(test), stores)
d["store_nbr"] = d["store_nbr"].astype("int8")


# Días festivos nacionales & Eventos
#d = pd.merge(d, events, how = "left")
d = pd.merge(d, national, how = "left")
# Regionales
d = pd.merge(d, regional, how = "left", on = ["date", "state"])
# Locales
d = pd.merge(d, local, how = "left", on = ["date", "city"])

# Día Laborales: Serán removidos cuándo la columna real de día laboral sea creada 
d = pd.merge(d,  work_day[["date", "type"]].rename({"type":"IsWorkDay"}, axis = 1),how = "left")


# Eventos
events["events"] = np.where(events.events.str.contains("futbol"), "Futbol", events.events)

def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = df.select_dtypes(["category", "object"]).columns.tolist()
    # categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    df.columns = df.columns.str.replace(" ", "_")
    return df, df.columns.tolist()

events, events_cat = one_hot_encoder(events, nan_as_category=False)
events["events_Dia_de_la_Madre"] = np.where(events.date == "2016-05-08", 1,events["events_Dia_de_la_Madre"])
events = events.drop(239)

d = pd.merge(d, events, how = "left")
d[events_cat] = d[events_cat].fillna(0)

# Caracteristicas
d["holiday_national_binary"] = np.where(d.holiday_national.notnull(), 1, 0)
d["holiday_local_binary"] = np.where(d.holiday_local.notnull(), 1, 0)
d["holiday_regional_binary"] = np.where(d.holiday_regional.notnull(), 1, 0)

# 
d["national_independence"] = np.where(d.holiday_national.isin(['Batalla de Pichincha',  'Independencia de Cuenca', 'Independencia de Guayaquil', 'Independencia de Guayaquil', 'Primer Grito de Independencia']), 1, 0)
d["local_cantonizacio"] = np.where(d.holiday_local.str.contains("Cantonizacio"), 1, 0)
d["local_fundacion"] = np.where(d.holiday_local.str.contains("Fundacion"), 1, 0)
d["local_independencia"] = np.where(d.holiday_local.str.contains("Independencia"), 1, 0)


holidays, holidays_cat = one_hot_encoder(d[["holiday_national","holiday_regional","holiday_local"]], nan_as_category=False)
d = pd.concat([d.drop(["holiday_national","holiday_regional","holiday_local"], axis = 1),holidays], axis = 1)

he_cols = d.columns[d.columns.str.startswith("events")].tolist() + d.columns[d.columns.str.startswith("holiday")].tolist() + d.columns[d.columns.str.startswith("national")].tolist()+ d.columns[d.columns.str.startswith("local")].tolist()
d[he_cols] = d[he_cols].astype("int8")

d[["family", "city", "state", "type"]] = d[["family", "city", "state", "type"]].astype("category")

del holidays, holidays_cat, work_day, local, regional, national, events, events_cat, tr, tr1, tr2, he_cols
gc.collect()

d.head(10)

C:\Users\PCREATHORS\AppData\Local\Temp\ipykernel_34192\3017511124.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  holidays = holidays.append(tr).reset_index(drop = True)
C:\Users\PCREATHORS\AppData\Local\Temp\ipykernel_34192\3017511124.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  holidays["description"] = holidays["description"].str.replace("-", "").str.replace("+", "").str.replace('\d+', '')
C:\Users\PCREATHORS\AppData\Local\Temp\ipykernel_34192\3017511124.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  holidays["description"] = holidays["description"].str.replace("-", "").str.replace("+", "").str.replace('\d+', '')
C:\Users\PCREATHORS\AppData\Local\Temp\ipykernel_34192\301751

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,holiday_local_Fundacion_de_Ibarra,holiday_local_Fundacion_de_Loja,holiday_local_Fundacion_de_Machala,holiday_local_Fundacion_de_Manta,holiday_local_Fundacion_de_Quito,holiday_local_Fundacion_de_Riobamba,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
6,6,2013-01-01,1,CELEBRATION,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
7,7,2013-01-01,1,CLEANING,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
8,8,2013-01-01,1,DAIRY,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
9,9,2013-01-01,1,DELI,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0


In [91]:
# Caracteristicas relacionadas a estacionalidades 

def create_date_features(df):
    df['month'] = df.date.dt.month.astype("int8")
    df['day_of_month'] = df.date.dt.day.astype("int8")
    df['day_of_year'] = df.date.dt.dayofyear.astype("int16")
    df['week_of_month'] = (df.date.apply(lambda d: (d.day-1) // 7 + 1)).astype("int8")
    df['week_of_year'] = (df.date.dt.weekofyear).astype("int8")
    df['day_of_week'] = (df.date.dt.dayofweek + 1).astype("int8")
    df['year'] = df.date.dt.year.astype("int32")
    df["is_wknd"] = (df.date.dt.weekday // 4).astype("int8")
    df["quarter"] = df.date.dt.quarter.astype("int8")
    df['is_month_start'] = df.date.dt.is_month_start.astype("int8")
    df['is_month_end'] = df.date.dt.is_month_end.astype("int8")
    df['is_quarter_start'] = df.date.dt.is_quarter_start.astype("int8")
    df['is_quarter_end'] = df.date.dt.is_quarter_end.astype("int8")
    df['is_year_start'] = df.date.dt.is_year_start.astype("int8")
    df['is_year_end'] = df.date.dt.is_year_end.astype("int8")
    # 0: Invierno - 1: Primavera - 2: Verano - 3: Otonio
    df["season"] = np.where(df.month.isin([12,1,2]), 0, 1)
    df["season"] = np.where(df.month.isin([6,7,8]), 2, df["season"])
    df["season"] = pd.Series(np.where(df.month.isin([9, 10, 11]), 3, df["season"])).astype("int8")
    return df
d = create_date_features(d)




# Columna día de trabajo
d["workday"] = np.where((d.holiday_national_binary == 1) | (d.holiday_local_binary==1) | (d.holiday_regional_binary==1) | (d['day_of_week'].isin([6,7])), 0, 1)
d["workday"] = pd.Series(np.where(d.IsWorkDay.notnull(), 1, d["workday"])).astype("int8")
d.drop("IsWorkDay", axis = 1, inplace = True)

# Días de pago en el sector público son pagados cada 15 días. En el 15 y día último de cada mes
# Súpermercados podrían verse afectados por esto.
d["wageday"] = pd.Series(np.where((d['is_month_end'] == 1) | (d["day_of_month"] == 15), 1, 0)).astype("int8")

d.head(15)

C:\Users\PCREATHORS\AppData\Local\Temp\ipykernel_34192\1874443887.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week_of_year'] = (df.date.dt.weekofyear).astype("int8")


,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,workday,wageday
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0
1,1,2013-01-01,1,BABY CARE,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0
2,2,2013-01-01,1,BEAUTY,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0
4,4,2013-01-01,1,BOOKS,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0
6,6,2013-01-01,1,CELEBRATION,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0
7,7,2013-01-01,1,CLEANING,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0
8,8,2013-01-01,1,DAIRY,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0
9,9,2013-01-01,1,DELI,0.0,0.0,Quito,Pichincha,D,13,...,1,1,0,1,0,1,0,0,0,0


In [92]:
a = d[(d.sales.notnull())].groupby(["date", "family"]).sales.mean().reset_index().set_index("date")

In [93]:
a = train.sort_values(["store_nbr", "family", "date"])
for i in [20, 30, 45, 60, 90, 120, 365, 730]:
    a["SMA"+str(i)+"_sales_lag16"] = a.groupby(["store_nbr", "family"]).rolling(i).sales.mean().shift(16).values
    a["SMA"+str(i)+"_sales_lag30"] = a.groupby(["store_nbr", "family"]).rolling(i).sales.mean().shift(30).values
    a["SMA"+str(i)+"_sales_lag60"] = a.groupby(["store_nbr", "family"]).rolling(i).sales.mean().shift(60).values
print("Correlation")
a[["sales"]+a.columns[a.columns.str.startswith("SMA")].tolist()].corr()

Correlation


,sales,SMA20_sales_lag16,SMA20_sales_lag30,SMA20_sales_lag60,SMA30_sales_lag16,SMA30_sales_lag30,SMA30_sales_lag60,SMA45_sales_lag16,SMA45_sales_lag30,SMA45_sales_lag60,...,SMA90_sales_lag60,SMA120_sales_lag16,SMA120_sales_lag30,SMA120_sales_lag60,SMA365_sales_lag16,SMA365_sales_lag30,SMA365_sales_lag60,SMA730_sales_lag16,SMA730_sales_lag30,SMA730_sales_lag60
sales,1.000000,0.907364,0.892963,0.872613,0.906646,0.894297,0.869580,0.907649,0.895976,0.869522,...,0.865390,0.902932,0.889616,0.863429,0.899855,0.889478,0.865063,0.898935,0.885013,0.854974
SMA20_sales_lag16,0.907364,1.000000,0.986043,0.948899,0.997391,0.983405,0.951559,0.992870,0.981195,0.952391,...,0.947832,0.980817,0.973423,0.944740,0.969820,0.967302,0.945562,0.970551,0.969151,0.941063
SMA20_sales_lag30,0.892963,0.986043,1.000000,0.962416,0.994382,0.997391,0.963505,0.996138,0.992870,0.965235,...,0.961737,0.985223,0.980817,0.958730,0.971537,0.969820,0.957514,0.970844,0.970551,0.956401
SMA20_sales_lag60,0.872613,0.948899,0.962416,1.000000,0.959120,0.975853,0.997391,0.976720,0.988893,0.992870,...,0.985446,0.988357,0.987671,0.980817,0.973777,0.972828,0.969820,0.971777,0.971455,0.970551
SMA30_sales_lag16,0.906646,0.997391,0.994382,0.959120,1.000000,0.991952,0.961490,0.997400,0.988719,0.962707,...,0.958872,0.985664,0.979272,0.955869,0.973798,0.971426,0.956389,0.972918,0.971730,0.953140
SMA30_sales_lag30,0.894297,0.983405,0.997391,0.975853,0.991952,1.000000,0.976045,0.997692,0.997400,0.976750,...,0.973593,0.989342,0.985664,0.970501,0.975362,0.973797,0.968486,0.973367,0.972918,0.968771
SMA30_sales_lag60,0.869580,0.951559,0.963505,0.997391,0.961490,0.976045,1.000000,0.977838,0.987773,0.997400,...,0.990029,0.991648,0.991224,0.985664,0.977580,0.976600,0.973797,0.974223,0.973926,0.972917
SMA45_sales_lag16,0.907649,0.992870,0.996138,0.976720,0.997400,0.997692,0.977838,1.000000,0.995813,0.978395,...,0.975046,0.990816,0.985734,0.972050,0.978180,0.976127,0.971833,0.975370,0.974343,0.970835
SMA45_sales_lag30,0.895976,0.981195,0.992870,0.988893,0.988719,0.997400,0.987773,0.995813,1.000000,0.986564,...,0.982019,0.993612,0.990816,0.978430,0.979599,0.978180,0.973673,0.975871,0.975370,0.972482
SMA45_sales_lag60,0.869522,0.952391,0.965235,0.992870,0.962707,0.976750,0.997400,0.978395,0.986564,1.000000,...,0.994535,0.994841,0.994880,0.990816,0.981663,0.980841,0.978180,0.976509,0.976345,0.975370


In [94]:
# Función para calcular caracteristicas de media móvil exponencial ponderada. Útil para capturar patrones de tendencia y suavizar datos antes de realizar un análisis o predicción.

def ewm_features(dataframe, alphas, lags):
    dataframe = dataframe.copy()
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby(["store_nbr", "family"])['sales']. \
                    transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe

alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [16, 30, 60, 90]

a = ewm_features(a, alphas, lags)

In [95]:
# Agregar caracteristicas de fechas en dataframe

a_df = a['month'] = a.date.dt.month.astype("int8")
a_df = a['year'] = a.date.dt.year.astype("int32")
a_df = a['week_of_month'] = (a.date.apply(lambda a: (a.day-1) // 7 + 1)).astype("int8")
a_df = a['week_of_year'] = a.date.dt.weekofyear.astype("int32")
a_df = a['season'] = np.where(a.month.isin([12,1,2]),0,1)
a_df = a['season'] = np.where(a.month.isin([6,7,8]),2,a["season"])
a_df = a['season'] = pd.Series(np.where(a.month.isin([9,10,11]),3,a["season"])).astype("int8")
a_df

C:\Users\PCREATHORS\AppData\Local\Temp\ipykernel_34192\2064384912.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  a_df = a['week_of_year'] = a.date.dt.weekofyear.astype("int32")


0          0
1          0
2          0
3          0
4          0
          ..
3000883    2
3000884    2
3000885    2
3000886    2
3000887    2
Length: 3000888, dtype: int8

In [96]:
# Reducción de datos a una sola tienda y producto

a_df  = a[(a['store_nbr'] == 1) & (a['family'] == "GROCERY I")]

In [97]:
a_df=a_df[['date',
      'sales']]

a_df.reset_index(inplace=True)

In [98]:
a_df

,index,date,sales
0,12,2013-01-01,0.0
1,1794,2013-01-02,2652.0
2,3576,2013-01-03,2121.0
3,5358,2013-01-04,2056.0
4,7140,2013-01-05,2216.0
...,...,...,...
1679,2991990,2017-08-11,1270.0
1680,2993772,2017-08-12,1630.0
1681,2995554,2017-08-13,952.0
1682,2997336,2017-08-14,2407.0


In [99]:
a_df=a_df.drop([0])

In [100]:
a_df.reset_index(inplace=True)

In [101]:
a_df['sales']

0       2652.0
1       2121.0
2       2056.0
3       2216.0
4        723.0
         ...  
1678    1270.0
1679    1630.0
1680     952.0
1681    2407.0
1682    2508.0
Name: sales, Length: 1683, dtype: float32

In [102]:
# Función simulación de precios

precio_inicial = 15
ventas_historicas = a_df['sales'] #df["ventas_historicas"]
lista_precios = [precio_inicial]

for i in range(len(ventas_historicas) -1):
  if( ventas_historicas[i] - ventas_historicas[i + 1]) > 0: #Vendes menos al siguiente día
    print("disminuyo ventas")
    #(precio del tiempo i) + (precio del tiempo i)*(porcentaje con probalidad de que salga un descuento)
    lista_precios.append(
      (lista_precios[-1] + lista_precios[-1] * np.random.choice( 
        (-.1, -.05, -0.03, -0.02, -0.01, 0, 0.01, 0.02, 0.03, .05, .1 ), 
        1, p = (.01, .05, .09, .1, .20, .40, 0.05, .04, 0.02, 0.02, 0.02)
        ))[0])
  else:
    print("aumento ventas")

    #(precio del tiempo i) + (precio del tiempo i)*(porcentaje con probalidad de que salga un aumento de precio)
    lista_precios.append(lista_precios[-1] + lista_precios[-1] * np.random.choice(
            (-.1, -.05, -0.03, -0.02, -0.01, 0, 0.01, 0.02, 0.03, .05, .1 ), 
            1,
            p = (0.02, 0.02, 0.02, 0.04, 0.05, 0.4, 0.2, 0.1, 0.09, 0.05, 0.01) 
         )[0]
    )

a_df["precios"] = lista_precios

disminuyo ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
aumento ventas
disminuyo ventas
aumento ventas
aumento ventas
aumento ventas
disminuyo ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
disminuyo ventas
aumento ventas
aumento ventas
aumento ventas
disminuyo ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
aumento ventas
aumento ventas
disminuyo ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento ventas
aumento ventas
disminuyo ventas
aumento ventas
disminuyo ventas
aumento vent

In [105]:
a_df=a_df[['date',
      'sales',
      'precios']]

a_df

,date,sales,precios
0,2013-01-02,2652.0,15.000000
1,2013-01-03,2121.0,14.550000
2,2013-01-04,2056.0,14.550000
3,2013-01-05,2216.0,14.695500
4,2013-01-06,723.0,14.695500
...,...,...,...
1678,2017-08-11,1270.0,44.645668
1679,2017-08-12,1630.0,44.645668
1680,2017-08-13,952.0,44.199211
1681,2017-08-14,2407.0,44.199211


In [107]:
simulacion_precios_ventas_reales = a_df

# Guardar el modelo en un archivo .pkl
with open('simulacion_precios.pkl', 'wb') as archivo:
    pickle.dump(simulacion_precios_ventas_reales, archivo)
